# Marelle Board import
* Getting players info dict => `MarelleBoard.players`
* 3 phases : place, move and end => `MarelleBoard.phase`
* Reset the board => `MarelleBoard.initialize_game()`
* Printing the board => `MarelleBoard.print_board()`
* Id:Action dict => `MarelleBoard.action_list_by_id`
* Action:Id dict => `MarelleBoard.action_id_by_action`
* Play an action => `MarelleBoard.play_action(action_id, player)`
* Get legal action ids => `MarelleBoard.get_legal_action_ids(player)`
* Check if game ended (returns 0 if not ended or winning player id) : `MarelleBoard.check_if_end()`

In [1]:
from maRL_board import MarelleBoard

In [ ]:
# Initialize board
board = MarelleBoard()

# Players place tokens
board.place_token(position=(0, 0), player=1)
board.place_token(position=(1, 0), player=2)
board.place_token(position=(2, 5), player=1)

# Players can move tokens
board.move_token(position=(0, 0), new_position=(0, 7), player=1)

# Print the board
board.print_board()



    0--------------0--------------0

    |              |              |

    |    0---------0---------0    |

    |    |         |         |    |

    |    |    0----0----0    |    |

    |    |    |         |    |    |

    0----0----0         0----2----0
                 
    |    |    |         |    |    |

    |    |    0----0----1    |    |

    |    |         |         |    |

    |    0---------0---------0    |

    |              |              |

    1--------------0--------------0
    


In [ ]:
# All token placement actions
print(board.place_token_action_list)
print(f"{len(board.place_token_action_list)} possible placement actions")

# All token movement actions
print(board.move_token_action_list)
print(f"{len(board.move_token_action_list)} possible movement actions")

# All player 1 legal token place actions
print(board.place_token_legal_actions(player=1))
print(f"{len(board.place_token_legal_actions(player=1))} legal placement actions")


# All player 1 legal token move actions
print(board.move_token_legal_actions(player=1))
print(f"{len(board.move_token_legal_actions(player=1))} legal movement actions")


[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7)]
24 possible placement actions
[((0, 0), (0, 1)), ((0, 1), (0, 0)), ((0, 0), (1, 0)), ((1, 0), (0, 0)), ((0, 0), (0, 7)), ((0, 7), (0, 0)), ((0, 1), (0, 2)), ((0, 2), (0, 1)), ((0, 2), (0, 3)), ((0, 3), (0, 2)), ((0, 2), (1, 2)), ((1, 2), (0, 2)), ((0, 3), (0, 4)), ((0, 4), (0, 3)), ((0, 4), (0, 5)), ((0, 5), (0, 4)), ((0, 4), (1, 4)), ((1, 4), (0, 4)), ((0, 5), (0, 6)), ((0, 6), (0, 5)), ((0, 6), (0, 7)), ((0, 7), (0, 6)), ((0, 6), (1, 6)), ((1, 6), (0, 6)), ((1, 0), (1, 1)), ((1, 1), (1, 0)), ((1, 0), (2, 0)), ((2, 0), (1, 0)), ((1, 0), (1, 7)), ((1, 7), (1, 0)), ((1, 1), (1, 2)), ((1, 2), (1, 1)), ((1, 2), (1, 3)), ((1, 3), (1, 2)), ((1, 2), (2, 2)), ((2, 2), (1, 2)), ((1, 3), (1, 4)), ((1, 4), (1, 3)), ((1, 4), (1, 5)), ((1, 5), (1, 4)), ((1, 4), (2, 4)), ((2, 4), (1, 4)), ((1, 5), (1, 6)), ((

In [ ]:
import gym
from gym import spaces


class MarelleGymEnv(gym.Env):
  """Custom Environment that follows gym interface"""
  metadata = {'render.modes': ['human']}

  def __init__(self):
    super(CustomEnv, self).__init__()    
    
    self.board = MarelleBoard()
    self.player = 1
    
    # Define action and observation space
    # They must be gym.spaces objects
    n_actions = len(self.board.place_token_action_list) + len(self.board.move_token_action_list)
    self.action_space = spaces.Discrete(n_actions)    # Example for using image as input:
    
    # the observation_space is the board 
    self.observation_space = spaces.Box(low=-1, high=1, shape=(3, 8), dtype=np.uint8)

  
  def step(self, action):
    # Execute one time step within the environment
  
  def reset(self):
    self.board = MarelleBoard()
  
  def render(self, mode='human', close=False):
    self.board.print_board()
